In [ ]:
### Intro to AI Group 3 Final Project - Using CV to Predict Ocular Disease

In [3]:
# These are the paths to the dataset folders on my local machine.
train_base_dir = "C:\\Users\\elanw\\OneDrive\\Documents\\IntroToAI\\ocular_dataset\\train"
image_dir = "C:\\Users\\elanw\\OneDrive\\Documents\\IntroToAI\\ocular_dataset\\preprocessed_images\\"

In [9]:
# Only run this once, to iterate through the examples in the csv, and reorganize the images into subfolders by label

import os
import shutil
import pandas as pd

ocular_data = pd.read_csv("C:\\Users\\elanw\\OneDrive\\Documents\\IntroToAI\\ocular_dataset\\full_df.csv")

categs = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']

if not os.path.exists(train_base_dir):
        os.mkdir(train_base_dir)

for categ in categs:
    if not os.path.exists(train_base_dir + "\\" + categ):
        os.mkdir(train_base_dir + "\\" + categ)


for index, row in ocular_data.iterrows():
    l_file = str(row['ID']) + "_left.jpg"
    r_file = str(row['ID'])+ "_right.jpg"
    if row['N'] == 1:
        dest = train_base_dir + "\\N\\"
    elif row['D'] == 1:
        dest = train_base_dir + "\\D\\"
    elif row['G'] == 1:
        dest = train_base_dir + "\\G\\"
    elif row['C'] == 1:
        dest = train_base_dir + "\\C\\"
    elif row['A'] == 1:
        dest = train_base_dir + "\\A\\"
    elif row['H'] == 1:
        dest = train_base_dir + "\\H\\"
    elif row['M'] == 1:
        dest = train_base_dir + "\\M\\"
    elif row['O'] == 1:
        dest = train_base_dir + "\\O\\"
    if os.path.exists(image_dir + l_file):
        shutil.copy(image_dir + l_file, dest + l_file)
    if os.path.exists(image_dir + r_file):
        shutil.copy(image_dir + r_file, dest + r_file)



In [ ]:
# This duplicates training data by creating a horizontally flipped version of each.

import cv2
import os

categs = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
for category in categs:
    for file in os.listdir(train_base_dir + "\\" + category):
        img = cv2.imread(train_base_dir + "\\" + category + "\\" + file)
        flipped_img = cv2.flip(img, 1)
        cv2.imwrite(train_base_dir + "\\" + category + "\\" + file[:-4] + "_flipped.jpg", flipped_img)

In [10]:
# This cell takes right images and flips them horizontally 
import cv2
import os

categs = ['N', 'D', 'G', 'C', 'A', 'H', 'M', 'O']
for category in categs:
    for file in os.listdir(train_base_dir + "\\" + category):
        if file.__contains__("right"):
            img = cv2.imread(train_base_dir + "\\" + category + "\\" + file)
            flipped_img = cv2.flip(img, 1)
            cv2.imwrite(train_base_dir + "\\" + category + "\\" + file[:-4] + ".jpg", flipped_img)

In [12]:
# Set up the base model for VGG16, using frozen pretrained weights from imagenet
# reducing the image size to 224x224

import keras
from tensorflow.keras.applications import VGG16
base_model = VGG16(weights='imagenet',
                   input_shape=(224, 224, 3),
                   include_top=False)
base_model.trainable = False

In [13]:
# add 2 dense layers and a prediction layer to the base model

from tensorflow.keras import models, layers
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(80, activation='relu')
dense_layer2 = layers.Dense(40, activation='relu')
prediction_layer = layers.Dense(8, activation='softmax')

xfer_vgg16_model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer,
    dense_layer2,
    prediction_layer
])

In [15]:
# Compile the model, using sparse categorical crossentropy rather than categorial
# because we have category labels that are exclusive from one another
# and data cannot be in more than one category
# Using Adam as the optimizer as experimentations running the model with SGD with various learning rates
# had inferior performance
xfer_vgg16_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [18]:
# Set up Image Data Generator
from tensorflow.keras.preprocessing.image import ImageDataGenerator
train_datagen = ImageDataGenerator(horizontal_flip=True, vertical_flip=False, validation_split=0.2)

In [19]:
# Flow images from directories with label data
# Setting class mode to sparse categorical for sparse categorical crossentropy
# Batch size 20-32 seems to perform well
# Shuffle set to false for validation data, as .labels and .classes returns labels in order, not matching shuffle status
# Keeping color mode rgb - vgg16 requires 3 input channels, and when using PIL and manually converting to grayscale,
# the model performance was not improved
train_it = train_datagen.flow_from_directory(train_base_dir, target_size=(224, 224), color_mode='rgb', class_mode='sparse', batch_size=32,  subset='training',  shuffle=True)
valid_it = train_datagen.flow_from_directory(train_base_dir, target_size=(224, 224), color_mode='rgb', class_mode='sparse', batch_size=32,  subset='validation', shuffle=False)

Found 5117 images belonging to 8 classes.
Found 1275 images belonging to 8 classes.


In [20]:
# Fit the model for 20 epochs
xfer_vgg16_model.fit(train_it, epochs=20, validation_data=valid_it)

Epoch 1/20
160/160 [==============================] - 444s 3s/step - loss: 3.7867 - accuracy: 0.3774 - val_loss: 2.2720 - val_accuracy: 0.3765
Epoch 2/20
160/160 [==============================] - 428s 3s/step - loss: 1.4919 - accuracy: 0.5212 - val_loss: 2.1253 - val_accuracy: 0.4000
Epoch 3/20
160/160 [==============================] - 414s 3s/step - loss: 1.1671 - accuracy: 0.5732 - val_loss: 1.8064 - val_accuracy: 0.4306
Epoch 4/20
160/160 [==============================] - 430s 3s/step - loss: 1.0514 - accuracy: 0.6058 - val_loss: 1.8944 - val_accuracy: 0.3788
Epoch 5/20
160/160 [==============================] - 415s 3s/step - loss: 0.9144 - accuracy: 0.6488 - val_loss: 1.9676 - val_accuracy: 0.3898
Epoch 6/20
160/160 [==============================] - 414s 3s/step - loss: 0.8555 - accuracy: 0.6811 - val_loss: 2.0284 - val_accuracy: 0.3843
Epoch 7/20
160/160 [==============================] - 453s 3s/step - loss: 0.7879 - accuracy: 0.6918 - val_loss: 2.0337 - val_accuracy: 0.4329

Above we can see the model continuing to overfit to the training data, while validation accuracy hovers around 40% without improving. These results seem to be consistent regardless of color or greyscale or whether using the SGD optimizer.

In [21]:
from sklearn.metrics import accuracy_score
predictions = xfer_vgg16_model.predict(valid_it)
score = accuracy_score(y_true=valid_it.classes, y_pred=predictions.argmax(axis=-1))
print(score)

40/40 [==============================] - 81s 2s/step
0.3819607843137255


In [23]:

import sklearn.metrics as metrics
print(metrics.classification_report(valid_it.classes, predictions.argmax(axis=-1)))

              precision    recall  f1-score   support

           0       0.24      0.11      0.15        53
           1       0.34      0.56      0.42        64
           2       0.43      0.47      0.45       424
           3       0.37      0.22      0.28        68
           4       0.00      0.00      0.00        17
           5       0.49      0.69      0.57        51
           6       0.47      0.31      0.37       420
           7       0.23      0.37      0.28       178

    accuracy                           0.38      1275
   macro avg       0.32      0.34      0.32      1275
weighted avg       0.40      0.38      0.38      1275



Above we can see that while the weakest performance is on the class with the fewest examples (4).

In [24]:
# Same attempt, but with VGG19
import keras
from tensorflow.keras.applications import VGG19
base_model = VGG16(weights='imagenet',
                   input_shape=(224, 224, 3),
                   include_top=False)
base_model.trainable = False
from tensorflow.keras import models, layers
flatten_layer = layers.Flatten()
dense_layer = layers.Dense(80, activation='relu')
dense_layer2 = layers.Dense(40, activation='relu')
prediction_layer = layers.Dense(8, activation='softmax')

xfer_vgg19_model = models.Sequential([
    base_model,
    flatten_layer,
    dense_layer,
    dense_layer2,
    prediction_layer
])
xfer_vgg19_model.compile(loss='sparse_categorical_crossentropy',optimizer='adam',metrics=['accuracy'])
xfer_vgg19_model.fit(train_it, epochs=20, validation_data=valid_it)

Epoch 1/20
160/160 [==============================] - 422s 3s/step - loss: 2.4235 - accuracy: 0.3311 - val_loss: 1.9876 - val_accuracy: 0.3451
Epoch 2/20
160/160 [==============================] - 418s 3s/step - loss: 1.8925 - accuracy: 0.3533 - val_loss: 1.9384 - val_accuracy: 0.3467
Epoch 3/20
160/160 [==============================] - 415s 3s/step - loss: 1.7782 - accuracy: 0.3682 - val_loss: 1.9625 - val_accuracy: 0.3631
Epoch 4/20
160/160 [==============================] - 420s 3s/step - loss: 1.7099 - accuracy: 0.3688 - val_loss: 1.8240 - val_accuracy: 0.3475
Epoch 5/20
139/160 [=========================>....] - ETA: 44s - loss: 1.6981 - accuracy: 0.3516

In [ ]:
# Show the accuracy score

from sklearn.metrics import accuracy_score
predictions = xfer_vgg19_model.predict(test_it)
score = accuracy_score(y_true=test_it.classes, y_pred=predictions.argmax(axis=-1))
print(score)

In [ ]:
# Display the classification report with precision, recall, and f1-score
# and number of examples for each category

import sklearn.metrics as metrics
print(metrics.classification_report(test_it.classes, predictions.argmax(axis=-1)))


In [ ]:
# Helpful function to show an image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

def show_image(image_path):
    image = mpimg.imread(image_path)
    plt.imshow(image)

show_image('C:\\Users\\elanw\\OneDrive\\Pictures\\eye_square.jpg')

In [ ]:
# Helper function to load image from path and preprocess it

from tensorflow.keras.preprocessing import image as image_utils
from tensorflow.keras.applications.vgg16 import preprocess_input

def load_and_process_image(image_path):
    img = image_utils.load_img(image_path, target_size=(224,224))
    img = image_utils.img_to_array(img)
    img = img.reshape(1,224,224,3)
    img = preprocess_input(img)
    return img